<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/earthengine-py-notebooks/tree/master/Join/inner_joins.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/earthengine-py-notebooks/blob/master/Join/inner_joins.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://mybinder.org/v2/gh/giswqs/earthengine-py-notebooks/master?filepath=Join/inner_joins.ipynb"><img width=58px src="https://mybinder.org/static/images/logo_social.png" />Run in binder</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/earthengine-py-notebooks/blob/master/Join/inner_joins.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## Install Earth Engine API
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geehydro](https://github.com/giswqs/geehydro). The **geehydro** Python package builds on the [folium](https://github.com/python-visualization/folium) package and implements several methods for displaying Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, `Map.centerObject()`, and `Map.setOptions()`.
The magic command `%%capture` can be used to hide output from a specific cell. Uncomment these lines if you are running this notebook for the first time.

In [1]:
# %%capture
# !pip install earthengine-api
# !pip install geehydro

Import libraries

In [2]:
import ee
import folium
import geehydro

Authenticate and initialize Earth Engine API. You only need to authenticate the Earth Engine API once. Uncomment the line `ee.Authenticate()` 
if you are running this notebook for the first time or if you are getting an authentication error.  

In [3]:
# ee.Authenticate()
ee.Initialize()

## Create an interactive map 
This step creates an interactive map using [folium](https://github.com/python-visualization/folium). The default basemap is the OpenStreetMap. Additional basemaps can be added using the `Map.setOptions()` function. 
The optional basemaps can be `ROADMAP`, `SATELLITE`, `HYBRID`, `TERRAIN`, or `ESRI`.

In [4]:
Map = folium.Map(location=[40, -100], zoom_start=4)
Map.setOptions('HYBRID')

## Add Earth Engine Python script 

In [5]:
# Make a date filter to get images in this date range.
dateFilter = ee.Filter.date('2014-01-01', '2014-02-01')

# Load a MODIS collection with EVI data.
mcd43a4 = ee.ImageCollection('MODIS/MCD43A4_006_EVI') \
    .filter(dateFilter)

# Load a MODIS collection with quality data.
mcd43a2 = ee.ImageCollection('MODIS/006/MCD43A2') \
    .filter(dateFilter)

# Define an inner join.
innerJoin = ee.Join.inner()

# Specify an equals filter for image timestamps.
filterTimeEq = ee.Filter.equals(**{
  'leftField': 'system:time_start',
  'rightField': 'system:time_start'
})

# Apply the join.
innerJoinedMODIS = innerJoin.apply(mcd43a4, mcd43a2, filterTimeEq)

# Display the join result: a FeatureCollection.
print('Inner join output:', innerJoinedMODIS)

# Map a function to merge the results in the output FeatureCollection.
joinedMODIS = innerJoinedMODIS.map(lambda feature: ee.Image.cat(feature.get('primary'), feature.get('secondary')))

# Print the result of merging.
print('Inner join, merged bands:', joinedMODIS.getInfo())



Inner join output: ee.FeatureCollection({
  "type": "Invocation",
  "arguments": {
    "join": {
      "type": "Invocation",
      "arguments": {},
      "functionName": "Join.inner"
    },
    "primary": {
      "type": "Invocation",
      "arguments": {
        "collection": {
          "type": "Invocation",
          "arguments": {
            "id": "MODIS/MCD43A4_006_EVI"
          },
          "functionName": "ImageCollection.load"
        },
        "filter": {
          "type": "Invocation",
          "arguments": {
            "rightField": "system:time_start",
            "leftValue": {
              "type": "Invocation",
              "arguments": {
                "start": "2014-01-01",
                "end": "2014-02-01"
              },
              "functionName": "DateRange"
            }
          },
          "functionName": "Filter.dateRangeContains"
        }
      },
      "functionName": "Collection.filter"
    },
    "secondary": {
      "type": "Invocation",
   

## Display Earth Engine data layers 

In [6]:
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map